# Hold My Data

<img src="img/python.svg" width="400px" style="margin-left: 0;" />

## Setup

In [25]:
import os

import dask.dataframe as ddf
import pandas as pd
import pyarrow as pa
import pyarrow.feather as feather
import pyarrow.parquet as pq

import sqlalchemy
import sqlalchemy_utils

from pandas_datapackage_reader import read_datapackage

In [2]:
pd.set_option('display.max_rows', 20)

## Data Source

In [3]:
df_orig = read_datapackage("https://github.com/openclimatedata/edgar-co2-emissions")
df_orig

Emissions
Code Name        Sector                      Year            
AFG  Afghanistan Transport                   1970   503.47130
                 Other industrial combustion 1970   128.94400
                 Buildings                   1970   641.58900
                 Non-combustion              1970   457.82630
                 Power Industry              1970    10.44530
ALB  Albania     Transport                   1970   726.04500
                 Other industrial combustion 1970  1472.33000
                 Buildings                   1970   992.68000
                 Non-combustion              1970   545.12550
                 Power Industry              1970   698.29100
...                                                       ...
ZMB  Zambia      Other industrial combustion 2016  1782.10400
                 Transport                   2016  1338.38500
                 Non-combustion              2016   838.28820
                 Buildings                   2016   229.69220
                 Power Industry              2016    42.11472
ZWE  Zimbabwe    Power Industry              2016  3019.65400
                 Other industrial combustion 2016  1847.38200
                 Buildings                   2016  2929.67800
                 Transport                   2016  1260.29300
                 Non-combustion              2016  1005.62100

[48598 rows x 1 columns]

In [4]:
df_orig = df_orig.reset_index()
df_orig['Bad'] = df_orig['Emissions'] > df_orig.groupby(['Year', 'Sector'])['Emissions'].transform('mean')
df_orig

,Code,Name,Sector,Year,Emissions,Bad
0,AFG,Afghanistan,Transport,1970,503.47130,False
1,AFG,Afghanistan,Other industrial combustion,1970,128.94400,False
2,AFG,Afghanistan,Buildings,1970,641.58900,False
3,AFG,Afghanistan,Non-combustion,1970,457.82630,False
4,AFG,Afghanistan,Power Industry,1970,10.44530,False
5,ALB,Albania,Transport,1970,726.04500,False
6,ALB,Albania,Other industrial combustion,1970,1472.33000,False
7,ALB,Albania,Buildings,1970,992.68000,False
8,ALB,Albania,Non-combustion,1970,545.12550,False
9,ALB,Albania,Power Industry,1970,698.29100,False


In [5]:
f"{df_orig.memory_usage(index=False, deep=True).sum() / 1024.**2:.2f}MB"

'10.08MB'

## CSV

<img src="img/csv.svg" width="200px" style="margin-left: 0;" />

- **Comma**-Separated Values
- "Standardized" in [RFC 4180](https://tools.ietf.org/html/rfc4180)
- No type markers

In [6]:
df_orig.to_csv('data/emissions.csv', index=False)
f"{os.stat('data/emissions.csv').st_size / 1024.**2:.2f}MB"

'2.42MB'

In [7]:
df_csv = pd.read_csv('data/emissions.csv')
df_csv

,Code,Name,Sector,Year,Emissions,Bad
0,AFG,Afghanistan,Transport,1970,503.47130,False
1,AFG,Afghanistan,Other industrial combustion,1970,128.94400,False
2,AFG,Afghanistan,Buildings,1970,641.58900,False
3,AFG,Afghanistan,Non-combustion,1970,457.82630,False
4,AFG,Afghanistan,Power Industry,1970,10.44530,False
5,ALB,Albania,Transport,1970,726.04500,False
6,ALB,Albania,Other industrial combustion,1970,1472.33000,False
7,ALB,Albania,Buildings,1970,992.68000,False
8,ALB,Albania,Non-combustion,1970,545.12550,False
9,ALB,Albania,Power Industry,1970,698.29100,False


### Data Preservation

[**be careful!**](https://stackoverflow.com/questions/50867308/pandas-read-csv-interprets-true-as-boolean-i-need-a-string)

> I'm reading stock tickers from comma delimited file using pandas.read_csv(). One of the tickers is TRUE, so pandas reader interprets it as a boolean...

In [8]:
df_csv.dtypes

Code          object
Name          object
Sector        object
Year           int64
Emissions    float64
Bad             bool
dtype: object

In [9]:
(df_orig == df_csv).describe()

,Code,Name,Sector,Year,Emissions,Bad
count,48598,48598,48598,48598,48598,48598
unique,1,1,1,1,2,1
top,True,True,True,True,True,True
freq,48598,48598,48598,48598,45311,48598


In [10]:
df_orig['Emissions'].values[-1], df_csv['Emissions'].values[-1]

(1005.621, 1005.6210000000001)

## Excel

<img src="img/docx.svg" width="200px" style="margin-left: 0;" />

- Office *Open* XML
- Standardized as [ISO/IEC 29500-1](https://www.iso.org/standard/71691.html) and [ECMA-376](https://www.ecma-international.org/publications/standards/Ecma-376.htm)
- MS Office 2007 **not compliant**, 2010 somehow is

In [169]:
excel_writer = pd.ExcelWriter('data/emissions.xlsx')
df_orig.to_excel(excel_writer, index=False)
excel_writer.save()
f"{os.stat('data/emissions.xlsx').st_size / 1024.**2:.2f}MB"

'1.46MB'

In [170]:
df_excel = pd.read_excel('data/emissions.xlsx')
df_excel

,Code,Name,Sector,Year,Emissions,Bad
0,AFG,Afghanistan,Transport,1970,503.47130,False
1,AFG,Afghanistan,Other industrial combustion,1970,128.94400,False
2,AFG,Afghanistan,Buildings,1970,641.58900,False
3,AFG,Afghanistan,Non-combustion,1970,457.82630,False
4,AFG,Afghanistan,Power Industry,1970,10.44530,False
5,ALB,Albania,Transport,1970,726.04500,False
6,ALB,Albania,Other industrial combustion,1970,1472.33000,False
7,ALB,Albania,Buildings,1970,992.68000,False
8,ALB,Albania,Non-combustion,1970,545.12550,False
9,ALB,Albania,Power Industry,1970,698.29100,False


### Data Preservation

In [173]:
idx = df_orig.index[~(df_orig['Emissions'] == df_excel['Emissions'])].values[0]
df_orig.loc[idx, 'Emissions'], df_excel.loc[idx, 'Emissions']

(4.3525800000000014, 4.352580000000001)

## SQL

<img src="img/postgresql.svg" width="200px" style="margin-left: 0;" />

- [Standarized in ISO/IEC 9075 up to SQL:2016](https://www.iso.org/standard/63555.html)
- [No full implementation available](https://en.wikipedia.org/wiki/SQL_compliance)
- PostgreSQL used as an example

In [155]:
engine = sqlalchemy.create_engine('postgresql+psycopg2://admin@localhost:5432/mydb', echo=False)
if not sqlalchemy_utils.database_exists(engine.url):
    sqlalchemy_utils.create_database(engine.url)

In [12]:
df_orig.to_sql('emissions', con=engine, if_exists='replace', index=True)

In [13]:
conn = engine.connect()
size = conn.execute("SELECT pg_size_pretty( pg_total_relation_size('emissions') );").fetchone()
conn.close()
size

('5704 kB',)

In [14]:
df_sql = pd.read_sql('select * from emissions order by index', con=engine)
df_sql

,index,Code,Name,Sector,Year,Emissions,Bad
0,0,AFG,Afghanistan,Transport,1970,503.47130,False
1,1,AFG,Afghanistan,Other industrial combustion,1970,128.94400,False
2,2,AFG,Afghanistan,Buildings,1970,641.58900,False
3,3,AFG,Afghanistan,Non-combustion,1970,457.82630,False
4,4,AFG,Afghanistan,Power Industry,1970,10.44530,False
5,5,ALB,Albania,Transport,1970,726.04500,False
6,6,ALB,Albania,Other industrial combustion,1970,1472.33000,False
7,7,ALB,Albania,Buildings,1970,992.68000,False
8,8,ALB,Albania,Non-combustion,1970,545.12550,False
9,9,ALB,Albania,Power Industry,1970,698.29100,False


### Data Preservation

In [15]:
(df_orig == df_sql.loc[:, df_orig.columns]).describe()

,Code,Name,Sector,Year,Emissions,Bad
count,48598,48598,48598,48598,48598,48598
unique,1,1,1,1,2,1
top,True,True,True,True,True,True
freq,48598,48598,48598,48598,45985,48598


In [16]:
idx = df_orig.index[~(df_orig['Emissions'] == df_sql['Emissions'])].values[0]
df_orig.loc[idx, 'Emissions'], df_sql.loc[idx, 'Emissions']

(4.3525800000000014, 4.35258)

## Arrow

<img src="img/arrow.png" width="200px" style="margin: 40px;" />

- Fast in-memory data format
- Stable type system

<img src="img/arrow_types.png" width="1200px" style="margin-left: 0; margin-top: 50px;" />

In [17]:
table_orig = pa.Table.from_pandas(df_orig)
table_orig

pyarrow.Table
Code: string
Name: string
Sector: string
Year: int64
Emissions: double
Bad: bool
metadata
--------
OrderedDict([(b'pandas',
              b'{"index_columns": [{"kind": "range", "name": null, "start": '
              b'0, "stop": 48598, "step": 1}], "column_indexes": [{"name": n'
              b'ull, "field_name": null, "pandas_type": "unicode", "numpy_ty'
              b'pe": "object", "metadata": {"encoding": "UTF-8"}}], "columns'
              b'": [{"name": "Code", "field_name": "Code", "pandas_type": "u'
              b'nicode", "numpy_type": "object", "metadata": null}, {"name":'
              b' "Name", "field_name": "Name", "pandas_type": "unicode", "nu'
              b'mpy_type": "object", "metadata": null}, {"name": "Sector", "'
              b'field_name": "Sector", "pandas_type": "unicode", "numpy_type'
              b'": "object", "metadata": null}, {"name": "Year", "field_name'
              b'": "Year", "pandas_type": "int64", "numpy_type": "int64", "m'
    

## Feather
- Arrow is just in-memory
- Feather adds thin on-disk container
- Similar to video formats (h264/h265/AV1 in MP4/AVI/Matroska)

In [18]:
feather.write_feather(df_orig, 'data/emissions.feather')
f"{os.stat('data/emissions.feather').st_size / 1024.**2:.2f}MB"

'2.67MB'

In [19]:
df_feather = feather.read_feather('data/emissions.feather')
df_feather

,Code,Name,Sector,Year,Emissions,Bad
0,AFG,Afghanistan,Transport,1970,503.47130,False
1,AFG,Afghanistan,Other industrial combustion,1970,128.94400,False
2,AFG,Afghanistan,Buildings,1970,641.58900,False
3,AFG,Afghanistan,Non-combustion,1970,457.82630,False
4,AFG,Afghanistan,Power Industry,1970,10.44530,False
5,ALB,Albania,Transport,1970,726.04500,False
6,ALB,Albania,Other industrial combustion,1970,1472.33000,False
7,ALB,Albania,Buildings,1970,992.68000,False
8,ALB,Albania,Non-combustion,1970,545.12550,False
9,ALB,Albania,Power Industry,1970,698.29100,False


### Data Preservation

In [20]:
(df_orig == df_feather).all().all()

True

### Data Exchange

In [21]:
df_feather2 = feather.read_feather('data/emissions2.feather')
df_feather2

,Code,Name,Sector,Year,Emissions,Emissions_sum
0,AFG,Afghanistan,Transport,1970,503.47130,153312.253780
1,AFG,Afghanistan,Other industrial combustion,1970,128.94400,153312.253780
2,AFG,Afghanistan,Buildings,1970,641.58900,153312.253780
3,AFG,Afghanistan,Non-combustion,1970,457.82630,153312.253780
4,AFG,Afghanistan,Power Industry,1970,10.44530,153312.253780
5,ALB,Albania,Transport,1970,726.04500,228309.948577
6,ALB,Albania,Other industrial combustion,1970,1472.33000,228309.948577
7,ALB,Albania,Buildings,1970,992.68000,228309.948577
8,ALB,Albania,Non-combustion,1970,545.12550,228309.948577
9,ALB,Albania,Power Industry,1970,698.29100,228309.948577


## Parquet

<img src="img/parquet.jpeg" width="400px" style="margin: 0;" />

- By Apache Foundation, specified [here](https://github.com/apache/parquet-format)
- Column-oriented
- Chunking ("row group")
- Physical and logical types (e.g. `date` is just `int32`, counting #days since 1st January 1970)
- Multiple compression levels:
  - Dictionay encoding
  - Run-length encoding
  - General purpose compression (uncompressed, Snappy, GZIP, LZO, Brotli, LZ4, ZSTD)

<img src="img/parquet_layout.gif" width="600px" style="margin-left: 0; margin-top: 40px;" />

In [22]:
pq.write_table(table_orig, 'data/emissions.parquet')
f"{os.stat('data/emissions.parquet').st_size / 1024.**2:.2f}MB"

'0.40MB'

In [23]:
table_parquet = pq.read_table('data/emissions.parquet')
df_parquet = table_parquet.to_pandas()

### Data Preservation

In [24]:
(df_orig == df_parquet).all().all()

True

## Dask

<img src="img/dask.svg" width="400px" style="margin-left: 0;" />


In [52]:
ddf_orig = ddf.from_pandas(df_orig, npartitions=1)
ddf_orig

,Code,Name,Sector,Year,Emissions,Bad
npartitions=1,,,,,,
0,object,object,object,int64,float64,bool
48597,...,...,...,...,...,...


In [151]:
ddf_orig.to_parquet('data/dask', partition_on=['Year'], engine='pyarrow')

In [152]:
ddf_recoverd = ddf.read_parquet('data/dask', engine='pyarrow')
ddf_recoverd

,Code,Name,Sector,Year,Emissions,Bad
npartitions=47,,,,,,
,object,object,object,int64,float64,bool
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [153]:
df_dask = ddf_recoverd.compute()
df_dask

,Code,Name,Sector,Year,Emissions,Bad
0,AFG,Afghanistan,Transport,1970,503.47130,False
1,AFG,Afghanistan,Other industrial combustion,1970,128.94400,False
2,AFG,Afghanistan,Buildings,1970,641.58900,False
3,AFG,Afghanistan,Non-combustion,1970,457.82630,False
4,AFG,Afghanistan,Power Industry,1970,10.44530,False
5,ALB,Albania,Transport,1970,726.04500,False
6,ALB,Albania,Other industrial combustion,1970,1472.33000,False
7,ALB,Albania,Buildings,1970,992.68000,False
8,ALB,Albania,Non-combustion,1970,545.12550,False
9,ALB,Albania,Power Industry,1970,698.29100,False


### Data Preservation

In [154]:
(df_orig.sort_values(['Year', 'Name', 'Sector']).reset_index(drop=True) == df_dask.sort_values(['Year', 'Name', 'Sector']).reset_index(drop=True)).all().all()

True

## Others

### HDF5

<img src="img/hdf.svg" width="200px" style="margin-left: 0;" />

- [Open standard](https://www.hdfgroup.org/)
- Large Community
- ... but [would not recommend it](https://cyrille.rossant.net/moving-away-hdf5/)

### SQLite

<img src="img/sqlite.svg" width="200px" style="margin-left: 0;" />

- [Single implementation](https://www.sqlite.org/)
- Embedded DB
- Nice format for small amounts of data
- Not good for large amounts

### JSON

<img src="img/json.svg" width="200px" style="margin-left: 0;" />

- [Web standard](https://www.json.org/)
- Human-readable
- Does not scale
- Nested, not tabular

### MessagePack

<img src="img/msgpack.png" width="200px" style="margin-left: 0; background: #292929; padding: 10px;" />

- [Nice, interopt standard](https://msgpack.org/)
- Binary JSON
- Quiet fast, still not fast enough

### Pickle
- [Python-only](https://docs.python.org/3/library/pickle.html)
- NOT safe!
- Does not survive library updates
- Slow
- Sometimes good for trusted IPC, bad for preservation

### More...
- [FlatBuffers](https://google.github.io/flatbuffers/)
- [Apache Thrift](https://thrift.apache.org/)
- [Protocol Buffers](https://developers.google.com/protocol-buffers/)
- [Apache Avro](https://avro.apache.org/)
- [Cap’n Proto](https://capnproto.org/)
- [Apache ORC](https://orc.apache.org/)
- **Stay open-minded!**